Valid:
* TOB3_long2
* TOB3_long3
* TOB3_long4

In [1]:
from unittest import TestCase
from math import ceil
from pathlib import Path
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import camp2ascii.formats as fmt
from camp2ascii import camp2ascii

fmt.REPAIR_MISALIGNED_MINOR_FRAMES = False


parent = Path("/home/alextsfox/git-repos/camp2ascii/tests")

in_dir = parent / "raw"

out_dir = parent / "c2a"
out_dir.mkdir(parents=True, exist_ok=True)

# fig, axs = plt.subplots(ceil(20/2), 2, figsize=(12, 15), sharex=True)
try:
    out_files = camp2ascii(in_dir, out_dir, pbar=True, verbose=3)
    for f in out_files:
        file_type = re.search(r"TOB\d", f.name).group(0)

        my_tob3 = pd.read_csv(f, skiprows=[0, 2, 3], na_values=["NAN", '"NAN"'])
        my_tob3["TIMESTAMP"] = pd.to_datetime(my_tob3["TIMESTAMP"], format="ISO8601")

        ref_file = list((out_dir.parent / "cc").glob(f"*{f.stem}*"))[0]
        ref_tob3 = pd.read_csv(ref_file, skiprows=[0, 2, 3], na_values=["NAN", '"NAN"'])
        ref_tob3["TIMESTAMP"] = pd.to_datetime(ref_tob3["TIMESTAMP"], format="ISO8601")

        if "temp_TMx(1)" in ref_tob3.columns:
            ref_tob3["temp_TMx(1)"] = pd.to_datetime(ref_tob3["temp_TMx(1)"], format="ISO8601")
            my_tob3["temp_TMx(1)"] = pd.to_datetime(my_tob3["temp_TMx(1)"], format="ISO8601")
        
        for col in ref_tob3.columns:
            if col in {"TIMESTAMP", "temp_TMx(1)"}:
                ref_tob3[col] = ref_tob3[col].astype(np.int64)
                my_tob3[col] = my_tob3[col].astype(np.int64)
            ref_tob3[col] = ref_tob3[col].astype(np.float64)
            my_tob3[col] = my_tob3[col].astype(np.float64)

        ref_tob3 = ref_tob3.set_index("RECORD")
        my_tob3 = my_tob3.set_index("RECORD")


        common_idx = ref_tob3.index.union(my_tob3.index)
        if file_type == "TOB3" and ref_tob3["TIMESTAMP"].diff().diff().abs().max() > 1e5:  # 100us
            common_idx = ref_tob3.index.intersection(my_tob3.index)
        ref_tob3 = ref_tob3.loc[common_idx].sort_index()
        my_tob3 = my_tob3.loc[common_idx].sort_index()

        print(np.allclose(ref_tob3, my_tob3, equal_nan=True))
finally:
    for f in out_files:
        f.unlink(missing_ok=True)

Processing files:  31%|███▏      | 221k/708k [00:00<00:00, 1.08MB/s] 

huh?
huh?
huh?
huh?
huh?
huh?


Processing files:  61%|██████    | 430k/708k [00:00<00:00, 1.04MB/s]

huh?
huh?
huh?
huh?
huh?
huh?


Processing files:  87%|████████▋ | 615k/708k [00:00<00:00, 954kB/s] 

...

Processing files: 865kB [00:00, 999kB/s]                           

huh?
huh?
huh?
huh?
huh?
huh?


Processing files: 100%|██████████| 708k/708k [00:01<00:00, 650kB/s]

huh?
huh?
huh?
huh?


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [56]:
def parse_footer(footer_bytes: bytes):
    content = int.from_bytes(footer_bytes, "little", signed=True)
    return dict(
        offset = content & 0x7FF,
        file_mark = bool((content >> 11) & 0x1),
        ring_mark = bool((content >> 12) & 0x1),
        empty_frame = bool((content >> 13) & 0x1),
        minor_frame = bool((content >> 14) & 0x1),
        validation = (content >> 16) & 0xFFFF,
    )
def parse_header(header_bytes: bytes):
    FRAME_HEADER_DTYPE = np.dtype([('seconds', '<i4'), ('subseconds', '<i4'), ('beg_record', '<i4')])
    return np.frombuffer(header_bytes, dtype=FRAME_HEADER_DTYPE)

In [ ]:
def parse_minor_frames(input_buffer: bytes, offset: int, frame_nbytes: int):
    """A minor frame flag indicates that the frame is split up into multiple sub-frames.
    The offset of the footer of each sub-frame gives the position of it's start byte within the major frame.
    The last sub-frame is always corrupted, so we start from the second to last one and work our way backwards until we end up at the start of the major frame.
    We then put the sub-frames in chronological order.

    The input_buffer should start at the very end of the major frame.
    """
    # the last minor frame is always corrupted, so we rewind to the previous one
    f.seek(-offset - 4, 1)
    minor_frame_nbytes = parse_footer(f.read(4))["offset"]

    minor_frame_footers_raw = []
    minor_frame_headers_raw = []
    minor_frame_data_raw = []
    while f.tell() > start:
        # go back to the head of the minor frame we sit at the foot of
        f.seek(-minor_frame_nbytes, 1)
        minor_frame_headers_raw.append(f.read(12))
        minor_frame_data_raw.append(f.read(minor_frame_nbytes - 16))
        minor_frame_footers_raw.append(f.read(4))
        
        # determine the sizee of the previous minor frame by reading its footer
        f.seek(-minor_frame_nbytes-4, 1)
        minor_frame_nbytes = parse_footer(f.read(4))["offset"]

        print(parse_header(minor_frame_headers_raw[-1]), parse_footer(minor_frame_footers_raw[-1]), len(minor_frame_data_raw[-1])+16)
    
    # put frames in chronological order
    minor_frame_headers_raw = minor_frame_headers_raw[::-1]
    minor_frame_data_raw = minor_frame_data_raw[::-1]
    minor_frame_footers_raw = minor_frame_footers_raw[::-1]

In [81]:
# 1008
import numpy as np

with open("tests/partial_frame_test/raw/TOB3_partial1.dat", "rb") as f:
    # good_bytes = f.read(1008)
    f.seek(0x5c90)
    bad_bytes = f.read(1008)

    # second minor frame starts at 760

    # try this: get footer offset, then rewind by offset and parse that frame
    # then rewind to the footer of the previous frame and parse that frame, until we reach the first frame we encountered.
    # we KNOW there is a good frame starting at 760 bytes into the bad_bytes

    frame_nbytes = 1008
    data_nbytes = 1008 - 4 - 12
    data_nlines = 8
    line_nbytes = 124

    bad_footer = parse_footer(bad_bytes[-4:])
    bad_header = parse_header(bad_bytes[:12])
    bad_data = bad_bytes[12:-4]

    print(bad_header, bad_footer, len(bad_data)+16)
    if bad_footer["offset"] != 0:
        start = f.tell() - frame_nbytes
        
        # the last minor frame is always corrupted, so we rewind to the previous one
        f.seek(-bad_footer["offset"] - 4, 1)
        minor_frame_nbytes = parse_footer(f.read(4))["offset"]

        minor_frame_footers_raw = []
        minor_frame_headers_raw = []
        minor_frame_data_raw = []
        while f.tell() > start:
            # go back to the head of the minor frame we sit at the foot of
            f.seek(-minor_frame_nbytes, 1)
            minor_frame_headers_raw.append(f.read(12))
            minor_frame_data_raw.append(f.read(minor_frame_nbytes - 16))
            minor_frame_footers_raw.append(f.read(4))
            
            # determine the sizee of the previous minor frame by reading its footer
            f.seek(-minor_frame_nbytes-4, 1)
            minor_frame_nbytes = parse_footer(f.read(4))["offset"]

            print(parse_header(minor_frame_headers_raw[-1]), parse_footer(minor_frame_footers_raw[-1]), len(minor_frame_data_raw[-1])+16)
        
        # put frames in chronological order
        minor_frame_headers_raw = minor_frame_headers_raw[::-1]
        minor_frame_data_raw = minor_frame_data_raw[::-1]
        minor_frame_footers_raw = minor_frame_footers_raw[::-1]

        print()
        for h, d, f in zip(minor_frame_headers_raw, minor_frame_data_raw, minor_frame_footers_raw):
            print(parse_header(h))
            print(d)
            print(parse_footer(f))
            print()

[(1140440850, 9250, 2102)] {'offset': 108, 'file_mark': False, 'ring_mark': False, 'empty_frame': False, 'minor_frame': True, 'validation': 46432} 1008
[(1140440850, 9600, 2108)] {'offset': 140, 'file_mark': False, 'ring_mark': False, 'empty_frame': False, 'minor_frame': False, 'validation': 46432} 140
[(1140440850, 9250, 2102)] {'offset': 760, 'file_mark': False, 'ring_mark': False, 'empty_frame': False, 'minor_frame': False, 'validation': 46432} 760

[(1140440850, 9250, 2102)]
b"64291\x00\x00\x00the quick brown fox jumped over the lazy dog\x00\x00\x00\x00why'd you leave the orange dish rag in the sink? It'll get mold!\x00\x00\x00\x0064291\x00\x00\x00the quick brown fox jumped over the lazy dog\x00\x00\x00\x00why'd you leave the orange dish rag in the sink? It'll get mold!\x00\x00\x00\x0064291\x00\x00\x00the quick brown fox jumped over the lazy dog\x00\x00\x00\x00why'd you leave the orange dish rag in the sink? It'll get mold!\x00\x00\x00\x0064291\x00\x00\x00the quick brown fox jumped